<img src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png" align=left alt="CC BY license logo" /><br /><br />
Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
**For questions/comments/improvements, email nathan.kelber@ithaka.org.**<br />
____

# Exploring Metadata and Pre-Processing

**Description of methods in this notebook:**
This [notebook](https://docs.tdm-pilot.org/key-terms/#jupyter-notebook) shows how to explore and pre-process the [metadata](https://docs.tdm-pilot.org/key-terms/#metadata) of a [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) using [Pandas](https://docs.tdm-pilot.org/key-terms/#pandas). 

The following processes are described:

* Importing a [CSV file](https://docs.tdm-pilot.org/key-terms/#csv-file) containing the [metadata](https://docs.tdm-pilot.org/key-terms/#metadata) for a given dataset ID
* Creating a [Pandas](https://docs.tdm-pilot.org/key-terms/#pandas) dataframe to view the [metadata](https://docs.tdm-pilot.org/key-terms/#metadata)
* Pre-processing your [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) by filtering out unwanted texts
* Exporting a list of relevant IDs to a [CSV file](https://docs.tdm-pilot.org/key-terms/#csv-file)
* Visualizing the metadata of your pre-processed [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) by the number of documents/year and pages/year

**Use Case:** For Learners (Detailed explanation, not ideal for researchers)

[Take me to the **Research Version** of this notebook ->](./exploring-metatadata-for-research)

**Difficulty:** Intermediate

**Completion time:** 45 minutes

**Knowledge Required:** 
* Python Basics Series ([Start Python Basics I](./python-basics-1.ipynb))

**Knowledge Recommended:**

* [Pandas I](./pandas-1.ipynb)

**Data Format:** [CSV file](https://docs.tdm-pilot.org/key-terms/#csv-file)

**Libraries Used:**
* [tdm_client](https://docs.tdm-pilot.org/key-terms/#tdm-client) to retrieve the [metadata](https://docs.tdm-pilot.org/key-terms/#metadata) in a [CSV file](https://docs.tdm-pilot.org/key-terms/#csv-file)
* [Pandas](https://docs.tdm-pilot.org/key-terms/#pandas) to manipulate and visualize the metadata

**Research Pipeline:** None
____

## Import your dataset

We'll use the `tdm_client` library to automatically retrieve the [metadata](https://docs.tdm-pilot.org/key-terms/#metadata) for a [dataset](https://docs.tdm-pilot.org/key-terms/#dataset). We can retrieve [metadata](https://docs.tdm-pilot.org/key-terms/#metadata) in a [CSV file](https://docs.tdm-pilot.org/key-terms/#csv-file) using the `get_metadata` method.

Enter a [dataset ID](https://docs.tdm-pilot.org/key-terms/#dataset-ID) in the next code cell. 

If you don't have a dataset ID, you can:
* Use the sample dataset ID already in the code cell
* [Create a new dataset](https://tdm-pilot.org/builder)
* [Use a dataset ID from other pre-built sample datasets](https://tdm-pilot.org/dataset/dashboard)

In [1]:
# Creating a variable `dataset_id` to hold our dataset ID
# The default dataset is Shakespeare Quarterly, 1950-present
dataset_id = "7e41317e-740f-e86a-4729-20dab492e925"

Next, import the `tdm_client`, passing the `dataset_id` as an argument using the `get_metadata` method.

In [2]:
# Import the `tdm_client`
import tdm_client

# Pull in our dataset CSV using
# The .get_metadata() method downloads the CSV file for our metadata
# to the /data folder and returns a string for the file name and location
# dataset_metadata will be a string containing that file name and location
dataset_metadata = tdm_client.get_metadata(dataset_id)

INFO:root:Metdata file /Users/test/data/7e41317e-740f-e86a-4729-20dab492e925.csv exists. Not re-downloading.


We are ready to import pandas for our analysis and create a dataframe. We will use the `read_csv()` method to create our dataframe from the CSV file.

In [6]:
# Import pandas 
import pandas as pd

# Create our dataframe
df = pd.read_csv(dataset_metadata)

We can confirm the size of our dataset using the `len()` function on our dataframe.

In [7]:
original_document_count = len(df)
print(f'Total original documents: {original_document_count}')

Total original documents: 6745


Now let's take a look at the data in our dataframe `df`. We will set pandas to show all columns using `set_option()` then get a preview using `head()`.

In [ ]:
# Set the pandas option to show all columns
# Setting None gives us all columns
# To show less columns replace None with an integer
pd.set_option("max_columns", None) 

# Show the first five rows of our dataframe
# To show a different number of preview rows
# Pass an integer into the .head()
df.head() 

Here are descriptions for the metadata types found in each column:

|Column Name|Description|
|---|---|
|id|a unique item ID (In JSTOR, this is a stable URL)|
|title|the title for the item|
|isPartOf|the larger work that holds this title (for example, a journal title)|
|publicationYear|the year of publication|
|doi|the digital object identifier for an item|
|docType|the type of document (for example, article or book)|
|provider|the source or provider of the dataset|
|datePublished|the publication date in yyyy-mm-dd format|
|issueNumber|the issue number for a journal publication|
|volumeNumber|the volume number for a journal publication|
|url|a URL for the item and/or the item's metadata|
|creator|the author or authors of the item|
|publisher|the publisher for the item|
|language|the language or languages of the item (eng is the ISO 639 code for English)|
|pageStart|the first page number of the print version|
|pageEnd|the last page number of the print version|
|placeOfPublication|the city of the publisher|
|wordCount|the number of words in the item|
|pageCount|the number of print pages in the item|
|outputFormat|what data is available ([unigrams](https://docs.tdm-pilot.org/key-terms/#unigram), [bigrams](https://docs.tdm-pilot.org/key-terms/#bigram), [trigrams](https://docs.tdm-pilot.org/key-terms/#trigram), and/or full-text)|

___

## Filtering out columns using Pandas

If there are any columns you would like to drop from your analysis, you can drop them with:

`df = df.drop(['column_name1', 'column_name2', ...], axis=1)`

In [ ]:
# Drop each of these named columns
# axis=1 specifies we are dropping columns
# axis=0 would specify to drop rows
df = df.drop(['outputFormat', 'pageEnd', 'pageStart', 'datePublished', 'language'], axis=1)

# Show the first five rows of our updated dataframe
df.head()

## Filtering out rows with Pandas

Now that we have filtered out unwanted metadata columns, we can begin filtering out any texts that may not match our research interests. Let's examine the first and last ten rows of the dataframe to see if we can identify texts that we would like to remove. We are looking for patterns in the metadata that could help us remove many texts at once.

In [15]:
# Preview the first ten items in the dataframe
# Can you identify patterns to select rows to remove?
df.head(10)

,id,title,isPartOf,publicationYear,doi,docType,provider,datePublished,issueNumber,volumeNumber,url,creator,publisher,language,pageStart,pageEnd,placeOfPublication,wordCount,pageCount,outputFormat
0,http://www.jstor.org/stable/2869669,In Praise of Two Eminent Shakespeareans,Shakespeare Quarterly,1979,NaN,article,jstor,1979-01-01,1.0,30,http://www.jstor.org/stable/2869669,A. B. Kernan,Folger Shakespeare Library,eng,100,103,NaN,1959,4,unigram; bigram; trigram
1,http://www.jstor.org/stable/2871200,Shakespeare In Edinburgh,Shakespeare Quarterly,1985,NaN,article,jstor,1985-07-01,2.0,36,http://www.jstor.org/stable/2871200,Gerald M. Berkowitz,Folger Shakespeare Library,eng,222,226,NaN,2961,5,unigram; bigram; trigram
2,http://www.jstor.org/stable/2871374,Subject Index,Shakespeare Quarterly,1997,NaN,article,jstor,1997-01-01,5.0,48,http://www.jstor.org/stable/2871374,NaN,Folger Shakespeare Library,eng,840,848,NaN,6128,9,unigram; bigram; trigram
3,http://www.jstor.org/stable/2868126,Shakespeare and Francis Sabie,Shakespeare Quarterly,1964,NaN,article,jstor,1964-10-01,4.0,15,http://www.jstor.org/stable/2868126,Ann Stanford,Folger Shakespeare Library,eng,454,455,NaN,636,2,unigram; bigram; trigram
4,http://www.jstor.org/stable/2871404,Review Article,Shakespeare Quarterly,1997,NaN,article,jstor,1997-04-01,1.0,48,http://www.jstor.org/stable/2871404,M. Lindsay Kaplan,Folger Shakespeare Library,eng,91,92,NaN,1011,2,unigram; bigram; trigram
5,http://www.jstor.org/stable/2866457,Review Article,Shakespeare Quarterly,1956,NaN,article,jstor,1956-04-01,NaN,2,http://www.jstor.org/stable/2866457,R. A. Foakes,Folger Shakespeare Library,eng,270,272,NaN,1098,3,unigram; bigram; trigram
6,http://www.jstor.org/stable/2867247,Review Article,Shakespeare Quarterly,1958,NaN,article,jstor,1958-04-01,2.0,9,http://www.jstor.org/stable/2867247,Edward Hubler,Folger Shakespeare Library,eng,192,193,NaN,654,2,unigram; bigram; trigram
7,http://www.jstor.org/stable/2868641,Reviews of Books Previously Recorded in Shakes...,Shakespeare Quarterly,1973,NaN,article,jstor,1973-10-01,4.0,24,http://www.jstor.org/stable/2868641,Bruce Nesbitt,Folger Shakespeare Library,eng,564,574,NaN,4875,11,unigram; bigram; trigram
8,http://www.jstor.org/stable/2868110,Review Article,Shakespeare Quarterly,1964,NaN,article,jstor,1964-10-01,4.0,15,http://www.jstor.org/stable/2868110,Robert Loper,Folger Shakespeare Library,eng,433,434,NaN,647,2,unigram; bigram; trigram
9,http://www.jstor.org/stable/2870020,"American Shakespeare Theatre, Stratford, Conne...",Shakespeare Quarterly,1981,NaN,article,jstor,1981-07-01,2.0,32,http://www.jstor.org/stable/2870020,Peter Saccio,Folger Shakespeare Library,eng,193,196,NaN,2606,4,unigram; bigram; trigram


In [16]:
# Preview the last ten items in the dataframe
# Can you identify patterns to select rows to remove?
df.tail(10)

,id,title,isPartOf,publicationYear,doi,docType,provider,datePublished,issueNumber,volumeNumber,url,creator,publisher,language,pageStart,pageEnd,placeOfPublication,wordCount,pageCount,outputFormat
6735,http://www.jstor.org/stable/2866881,The Rape of Tarquin,Shakespeare Quarterly,1959,NaN,article,jstor,1959-07-01,3.0,10,http://www.jstor.org/stable/2866881,Sam Hynes,Folger Shakespeare Library,eng,451,453,NaN,1131,3,unigram; bigram; trigram
6736,http://www.jstor.org/stable/2868432,[Illustration]: Music for the Chorus Between A...,Shakespeare Quarterly,1970,NaN,article,jstor,1970-10-01,4.0,21,http://www.jstor.org/stable/2868432,NaN,Folger Shakespeare Library,eng,456,456,NaN,194,1,unigram; bigram; trigram
6737,http://www.jstor.org/stable/2869797,Shakespeare in Ireland,Shakespeare Quarterly,1982,NaN,article,jstor,1982-07-01,2.0,33,http://www.jstor.org/stable/2869797,Christopher Murray,Folger Shakespeare Library,eng,197,199,NaN,1510,3,unigram; bigram; trigram
6738,http://www.jstor.org/stable/24778461,Review Article,Shakespeare Quarterly,2013,NaN,article,jstor,2013-07-01,2.0,64,http://www.jstor.org/stable/24778461,Frank Nicholas Clary,Folger Shakespeare Library,eng,244,246,NaN,1416,3,unigram; bigram; trigram
6739,http://www.jstor.org/stable/2871326,Romances,Shakespeare Quarterly,1997,NaN,article,jstor,1997-01-01,5.0,48,http://www.jstor.org/stable/2871326,NaN,Folger Shakespeare Library,eng,601,601,NaN,307,1,unigram; bigram; trigram
6740,http://www.jstor.org/stable/2869020,The Idea of Excellence in Shakespeare,Shakespeare Quarterly,1976,NaN,article,jstor,1976-04-01,2.0,27,http://www.jstor.org/stable/2869020,Madeleine Doran,Folger Shakespeare Library,eng,133,149,NaN,7737,17,unigram; bigram; trigram
6741,http://www.jstor.org/stable/2867945,[Illustration]: Mountebanks performing in the ...,Shakespeare Quarterly,1964,NaN,article,jstor,1964-01-01,1.0,15,http://www.jstor.org/stable/2867945,NaN,Folger Shakespeare Library,eng,NaN,NaN,NaN,55,1,unigram; bigram; trigram
6742,http://www.jstor.org/stable/2867795,Shakesepeare's Richard III and St. Paul,Shakespeare Quarterly,1963,NaN,article,jstor,1963-04-01,2.0,14,http://www.jstor.org/stable/2867795,Geoffrey Carnall,Folger Shakespeare Library,eng,186,188,NaN,1154,3,unigram; bigram; trigram
6743,http://www.jstor.org/stable/2868760,Duke Vincentio and the Illusion of Comedy or A...,Shakespeare Quarterly,1971,NaN,article,jstor,1971-01-01,1.0,22,http://www.jstor.org/stable/2868760,Hal Gelb,Folger Shakespeare Library,eng,25,34,NaN,5241,10,unigram; bigram; trigram
6744,http://www.jstor.org/stable/2868335,Review Article,Shakespeare Quarterly,1964,NaN,article,jstor,1964-07-01,3.0,15,http://www.jstor.org/stable/2868335,Calvin D. Linton,Folger Shakespeare Library,eng,235,236,NaN,681,2,unigram; bigram; trigram


### Remove all rows without data for a particular column

For example, we may wish to remove any texts that do not have authors. (In the case of journals, this may be helpful for removing paratextual sections such as the table of contents, indices, etc.) The column of interest in this case is `creator`. 

In [18]:
# Remove all texts without an author
df = df.dropna(subset=['creator']) #drop each row that has no value under 'creators'

In [19]:
# Print the total original documents followed by the current number
print(f'Total original documents: {original_document_count}')
print(f'Total current documents: ', len(df))

Total original documents: 6745
Total current documents:  5348


### Remove row based on the content of a particular column

We can also remove texts, depending on whether we do (or do not) want a particular value in a column. Here are a few examples.

In [27]:
# Remove all items with a particular title
# Change title to desired column
# Change `Review Article` to your undesired title

df = df[df.title != 'Review Article']
print('Total current documents ', len(df))

Total current documents  1647


In [23]:
# Keep only items with a particular language
# Change language to desired column
# Change 'eng' to your desired language

df = (df[df.language == 'eng']) # Change to another language code for other languages
print('Total current documents ', len(df))

In [21]:
# Remove all items with less than 3000 words
# Change wordCount to desired column
# Change '> 3000' to your desired expression to evaluate

df = df[df.wordCount > 1500]
print('Total current documents ', len(df))

In [22]:
# Print the total original documents followed by the current number
print('Total original documents:', original_document_count)
print('Total current documents: ', len(df))

Total original documents: 6745
Total current documents:  1712


Take a final look at your dataframe to make sure the current texts fit your research goals. In the next step, we will save the IDs of your pre-processed dataset.

In [ ]:
# Preview the first 50 rows of your dataset
# If all the items look good, move to the next step.
df.head(50)

## Saving a list of IDs to a CSV file

In [ ]:
# Write the column "id" to a CSV file called `pre-processed_###.csv` where ### is the `dataset_id`
df["id"].to_csv('data/pre-processed_' + dataset_id + '.csv')

Download the "pre-processed_###.csv" file (where ### is the `dataset_id`) for future analysis. You can use this file in combination with the dataset ID to automatically filter your texts and reduce the processing time of your analyses.

---
## Visualizing the Pre-Processed Data

In [ ]:
# For displaying plots
%matplotlib inline

In [ ]:
# Group the data by publication year and the aggregated number of ids into a bar chart
df.groupby(['publicationYear'])['id'].agg('count').plot.bar(title='Documents by year', figsize=(20, 5), fontsize=12); 

# Read more about Pandas dataframe plotting here: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html

And now let's look at the total page numbers by year.

In [ ]:
# Group the data by publication year and aggregated sum of the page counts into a bar chart

df.groupby(['publicationYear'])['pageCount'].agg('sum').plot.bar(title='Pages by decade', figsize=(20, 5), fontsize=12);